# Metaprogramación y macros

* [https://docs.julialang.org/en/v1/manual/metaprogramming/](https://docs.julialang.org/en/v1/manual/metaprogramming/)

* [https://en.wikibooks.org/wiki/Introducing_Julia/Metaprogramming](https://en.wikibooks.org/wiki/Introducing_Julia/Metaprogramming)

* [https://www.youtube.com/watch?v=2QLhw6LVaq0](https://www.youtube.com/watch?v=2QLhw6LVaq0)

* [https://www.youtube.com/watch?v=e6LGMeoQhfs](https://www.youtube.com/watch?v=e6LGMeoQhfs)

## Parseo

Todos los programas de Julia comienzan como una cadena de caracteres

In [ ]:
prog = "1+1"

Julia *parsea* estas cadenas de caracteres a *expresiones* (objetos de tipo `Expr`)

In [ ]:
expr1 = Meta.parse(prog)

In [ ]:
typeof(expr1)

También puede parsear a *símbolos* (objetos de tipo `Symbol`)

In [ ]:
expr2 = Meta.parse("x")

In [ ]:
typeof(expr2)

o a *constantes* tales como números, strings, etc.

In [ ]:
expr3 = Meta.parse("1")

In [ ]:
typeof(expr3)

## Expresiones

Las expresiones poseen dos partes. Una llamada `head` que típicamente contiene un símbolo que indica el tipo de la expresión

In [ ]:
expr1.head

y otra llamada `args` que suele ser un vector conteniendo otras expresiones, símbolos o contstantes

In [ ]:
expr1.args

Las expresiones también pueden ser construidas llamando a su constructor

In [ ]:
expr5 = Expr(:call,:+,1,1)

In [ ]:
expr1 == expr5

El punto clave es que en Julia, el código es internamente representado como una estructura de datos (la `struct Expr`) que es acesible desde el lenguaje mismo.

Esto permite a Julia manipular código Julia como si fuese cualquier otro tipo de datos (meta programación).

La función `dump` desgloza el contenido de una expresión

In [ ]:
dump(expr1)

Las expresiones puden anidarse

In [ ]:
dump(:(1+2*3))

Recordemos que el uso de operadores tales como `+`, `*`, etc. es transformado a correspondientes llamados a funciones

In [ ]:
+(1,*(2,3))

In [ ]:
dump(:(+(1,*(2,3))))

## Símbolos

Como ya dijimos, los símbolos son objetos de tipo `Symbol`. Un símbolo puede construirse vía su constructor

In [ ]:
Symbol("perro")

O usando el prefijo `:`

In [ ]:
:gato

## Quoting

Podemos decirle a Julia que no evalúe una expresión englobándola en un bloque `quote`

In [ ]:
quote
    1+1
end

In [ ]:
expr6 = quote
    1+1
end

In [ ]:
typeof(expr6)

In [ ]:
expr6.head

In [ ]:
expr6.args

Notar que los bloques `quote` contienen información sobre donde fué creada cada línea de código.

También podemos "quotear" una secuencia de expresiones

In [ ]:
quote
    1+1
    x=2
    for i in 1:10
        println("i=",i)
    end
end

Alternativamente, para expresiones de una sola línea, podemos "quotear" una expresión englobándola con el prefijo `:` (analogamente a como ocurre con los símbolos)

In [ ]:
:(1+1)

In [ ]:
:(x=2)

Remarcamos que expresiones equivalentes pueden ser construidas usando `Meta.parse` o el constructor `Expr`

In [ ]:
:(a + b*c + 1) ==
       Meta.parse("a + b*c + 1") ==
       Expr(:call, :+, :a, Expr(:call, :*, :b, :c), 1)

## Interpolación

Para facilitar la construcción de expresiones, Julia ofrece usar *interpolación* en expresiones, de manera similar a como lo hacemos con cadenas de caracteres

In [ ]:
a = 1

ex = :($a + b)

Otro ejemplo. La tupla `(1,2,3)` es interpolada dentro de una expresión condicional

In [ ]:
ex = :(a in $:((1,2,3)) )

Interpolar una expresión no quoteada causa error

In [ ]:
$a + b

Ejemplo: Algunas veces necesitamos interpolar un array de expresiones de manera que se conviertan en argumentos de otras, tal como cuando queremos generar una llamada a función de manera programática

In [ ]:
args = [:x, :y, :z]

:(f(1, $(args...)))

## Evaluando expresiones

Dada una expresión, podemos evaluarla (i.e. ejecutarla) en Julia usando la función `eval()`. Por ejemplo

In [ ]:
ex = :(1 + 2)
eval(ex)

De esta manera, es posible generar código en tiempo de ejecución y luego correrlo usando `eval()`. Por ejemplo

In [ ]:
a = 1
ex = Expr(:call, :+, a, :b)

In [ ]:
a = 0 
b = 2
eval(ex)

## Funciones sobre expresiones

Podemos definir funciones que tomen como entrada expresiones y devuelvan expresiones. Por ejemplo

In [ ]:
function math_expr(op, op1, op2)
    expr = Expr(:call, op, op1, op2)
    return expr
end

In [ ]:
ex = math_expr(:+, 1, Expr(:call, :*, 4, 5))

In [ ]:
eval(ex)

## Macros

Los macros se parecen a funciones que procesan expresiones, pero se evalúan de manera diferente.

Cuando uno llama a una función, los expresiones pasadas como argumentos son evaluadas antes del llamado

In [ ]:
function f(x)
   println("x = ",x) 
end
f(1+1)

Esto no sirve para definir funciones de expresiones en expresiones.

Podemos solucionar el problema "quoteando" los argumentos pasados a la función

In [ ]:
f(:(1+1))

Pero esto resulta engorroso. Es mejor introducir algo parecido a las funciones, pero que al llamarlas no resulte en la evaluación de las expresiones pasadas como argumentos.

Esta alternativa a las funciones la proveen los macros, los cuales se definen como

In [ ]:
macro m(x)
    println("x = ",x)
end

y se los llama usando `@` como prefijo y omitiendo paréntesis

In [ ]:
@m 1+1

Si por alguna razón deseamos eventualmente evaluar las expresiones pasadas como argumentos, podemos hacerlo dentro del macro combinando `quote` con interpolación vía el operador `$`

In [ ]:
macro m(x)
    quote
        println("x = ",$x)
    end
end

In [ ]:
@m 1+1

### Ejemplo 1

Veamos ejemplos útiles de macros que suelen usarse en Julia

In [ ]:
@elapsed rand(1000000)

Para entender mejor que es lo que ocurre, inspeccionemos lo que `elapsed` hace con la ayuda de otro macro llamado `macroexpand`

In [ ]:
@macroexpand @elapsed rand(1000000)

Vemos que `elapsed` retorna un bloque de código en donde la llamada a `rand` se encuentra entre dos llamadas a `Base.time_ns()` que se usan para contabilizar el tiempo transcurrido.

Si `elapsed` fuese una función, la llamada a `rand` ocurriría antes de evaluar el cuerpo de la misma, resultando en una diferencia de tiempo incorrecta.

### Ejemplo 2

`@time` sirve para cronometrar el tiempo requerido para evaluar una expresión

In [ ]:
j=0
@time for i in 1:100000000 j+=1 end

### Ejemplo 3

`@printf` print arguments usando el estilo de `printf` de `C` para la especificación de formatos de cadenas de caracteres (string).

Opcionalmente, se le puede pasar un `IO` como primer argumento para redirigir la salida (output).

* [https://docs.julialang.org/en/v1/stdlib/Printf/](https://docs.julialang.org/en/v1/stdlib/Printf/)

In [ ]:
using Printf 
@printf "Scientific notation %e" 1.234

In [ ]:
@printf "Scientific notation three digits %.3e" 1.23456

In [ ]:
 @printf "Padded with zeros to length 6 %06i" 123

### Ejemplo 4

`@benchmark` sirve para realizar un análisis más detallado de la evaluación de una expresión.

In [ ]:
using BenchmarkTools
@benchmark rand(1000000)

`@btime` es similar a `@time`, sólo que el primero descuenta el tiempo de compilación requerido.

In [ ]:
@btime rand(1000000)

### Ejemplo 5

`@view` sirve para crear *vistas de arrays*.

In [ ]:
t = rand(Float64,(3,4,5))

In [ ]:
a = @view t[2:3,2:end,5]

In [ ]:
a .= 0.0

In [ ]:
t